In [85]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import DBSCAN, OPTICS, KMeans
from sklearn import metrics
import hdbscan
import time
import umap
import umap.plot

In [98]:
# ! pip install pyarrow

In [99]:
df = pd.read_parquet('dataset/semantic_similarity/hscode.parquet')

In [100]:
df_hscode2022 = df[df['HSYear'] == 2022]

In [102]:
# df_hscode2022

In [4]:
# df['thdescriptions2']

In [90]:
# df[['thdescriptions4','HSCode2']]

In [109]:
df_hscode4_th = df[['thdescriptions4','HSCode2']]
df_hscode4_th = df_hscode4_th.rename(columns={'thdescriptions4': 'hscode4_text'})

df_hscode4_en = df[['endescriptions4','HSCode2']]
df_hscode4_en = df_hscode4_en.rename(columns={'endescriptions4': 'hscode4_text'})

df_hscode4 = pd.concat([
    df_hscode4_th,
    df_hscode4_en
])
df_hscode4 = df_hscode4.drop_duplicates()
df_hscode4 = df_hscode4[df_hscode4['hscode4_text'].notnull()]

In [111]:
# df_hscode4[df_hscode4['hscode4_text'].isnull()]

In [104]:
df_hscode4

,hscode4_text,HSCode2
0,ม้า ลา และล่อมีชีวิต,01
3,สัตว์จำพวกโคกระบือ มีชีวิต,01
8,สุกรมีชีวิต,01
11,แกะและแพะมีชีวิต,01
15,สัตว์ปีกเลี้ยงมีชีวิต ได้แก่ ไก่ชนิดแกลลัสโดเม...,01
...,...,...
56468,"Sanitary towels (pads) and tampons, napkins an...",96
56573,"Postage or revenue stamps, stamp-postmarks, fi...",97
67841,"Original engravings, prints and lithographs. ...",97
67919,Antiques of an age exceeding one hundred years...,97


In [25]:
# sentences_text = np.concatenate((
#     # df['thdescriptions2'].unique(), 
#     df['thdescriptions4'].unique(),
#     # df['endescriptions2'].unique(),
#     df['endescriptions4'].unique(),
# ))
# sentences_text.shape

In [26]:
# df_sentences = pd.DataFrame(sentences_text, columns=['text'])
# df_sentences.shape

In [27]:
# df_sentences.sample(20)

In [105]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [112]:
X = model.encode(df_hscode4['hscode4_text'].values, convert_to_tensor=True)

In [113]:
X.shape

torch.Size([2623, 512])

In [114]:
corpus_embeddings = X

In [121]:
start_time = time.time()
clusters = util.community_detection(corpus_embeddings, min_community_size=5, threshold=0.35)

# print("Clustering done after {:.2f} sec".format(time.time() - start_time))

# #Print for all clusters the top 3 and bottom 3 elements
# for i, cluster in enumerate(clusters):
#     print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
#     for sentence_id in cluster[0:3]:
#         print("\t", df_hscode4.iloc[sentence_id])
#     print("\t", "...")
#     for sentence_id in cluster[-3:]:
#         print("\t", df_hscode4.iloc[sentence_id])

In [122]:
df_hscode4.iloc[[0,2,3]]

,hscode4_text,HSCode2,cluster_group_id
0,ม้า ลา และล่อมีชีวิต,01,-1
8,สุกรมีชีวิต,01,-1
11,แกะและแพะมีชีวิต,01,-1


In [123]:
df_hscode4['cluster_group_id'] = -1

In [134]:
# df_hscode4['cluster_group_id'].value_counts()

In [178]:
def find_accuracy(series):    
    n = 1
    count_series = series.value_counts()
    sum_all = count_series.sum()
    count_series_no_noise = count_series[count_series.index != -1]
    sum_top_n = count_series_no_noise.iloc[:n].sum()
    accuracy = sum_top_n / sum_all
    # import pdb;pdb.set_trace()
    return accuracy

In [181]:
df_hscode4.groupby('HSCode2')['cluster_group_id'].apply(find_accuracy).mean()

0.33300416109584574

In [182]:
# count_series = df_hscode4[df_hscode4['HSCode2'] == '00']['cluster_group_id'].value_counts()
# count_series_no_noise = count_series[count_series.index != -1]
# sum_top_n = count_series_no_noise.iloc[:n].sum()
# accuracy = sum_top_n / sum_all
# accuracy

In [ ]:
(4) + (2) + 1 + 1 + 1 / 8

In [138]:
6 * 0.5  + 6 * 0.5 + = 6

4.0

In [137]:
12 / 12

1.0

In [176]:
df_hscode4[df_hscode4['HSCode2'] == '02']

,hscode4_text,HSCode2,cluster_group_id
48,เนื้อสัตว์จำพวกโค กระบือ สดหรือแช่เย็น,02,1
54,เนื้อสัตว์จำพวกโค กระบือ แช่แข็ง,02,78
60,เนื้อสุกร สด แช่เย็นหรือแช่แข็ง,02,11
66,เนื้อแกะหรือเนื้อแพะ สด แช่เย็นหรือแช่แข็ง,02,1
75,เนื้อม้า เนื้อลาหรือเนื้อล่อ สด แช่เย็น หรือแช...,02,0
76,ส่วนอื่นที่บริโภคได้ของสัตว์จำพวกโค กระบือ สุก...,02,0
85,เนื้อและส่วนอื่นที่บริโภคได้ของสัตว์ปีกเลี้ยงต...,02,0
107,เนื้อและส่วนอื่นที่บริโภคได้ของสัตว์อื่นๆ นอกจ...,02,0
115,มันหมูที่ไม่มีเนื้อติด และมันสัตว์ปีกเลี้ยง ยั...,02,0
116,เนื้อ และส่วนอื่นของสัตว์ที่บริโภคได้ ใส่เกลือ...,02,0


In [133]:
df_hscode4

,hscode4_text,HSCode2,cluster_group_id
0,ม้า ลา และล่อมีชีวิต,01,4
3,สัตว์จำพวกโคกระบือ มีชีวิต,01,78
8,สุกรมีชีวิต,01,60
11,แกะและแพะมีชีวิต,01,4
15,สัตว์ปีกเลี้ยงมีชีวิต ได้แก่ ไก่ชนิดแกลลัสโดเม...,01,63
...,...,...,...
56468,"Sanitary towels (pads) and tampons, napkins an...",96,2
56573,"Postage or revenue stamps, stamp-postmarks, fi...",97,17
67841,"Original engravings, prints and lithographs. ...",97,85
67919,Antiques of an age exceeding one hundred years...,97,-1


In [120]:
df_hscode4.sort_values(by='HSCode2')[:60]

,hscode4_text,HSCode2,cluster_group_id
82454,service business,00,-1
82454,ธุรกิจบริการ,00,-1
32,"Other live animals, other than live animals sp...",01,-1
15,"Live poultry, that is to say, fowls of the spe...",01,-1
11,Live sheep and goats.,01,-1
8,Live swine.,01,-1
3,Live bovine animals.,01,-1
0,"Live horses, asses, mules and hinnies.",01,-1
0,ม้า ลา และล่อมีชีวิต,01,-1
3,สัตว์จำพวกโคกระบือ มีชีวิต,01,-1


In [77]:
df_hscode4[df_hscode4['HSCode2'] == '48']

,hscode4_text,HSCode2,group_id,predict_group_id,cluster_group_id
98,กระดาษคราฟต์ ที่ไม่เคลือบ เป็นม้วนหรือเป็นแผ่น...,48,5,-1,-1
108,กระดาษและกระดาษแข็งที่ไม่เคลือบ ชนิดที่ใช้สำหร...,48,5,-1,-1
112,กระดาษชำระและกระดาษที่คล้ายกัน เซลลูโลสแวดดิ้ง...,48,5,-1,-1
292,กระสวย หลอด กรวย และสิ่งรองรับที่คล้ายกันสำหรั...,48,5,-1,-1
98,"Uncoated kraft paper and paperboard, in rolls ...",48,5,-1,-1
108,"Uncoated paper and paperboard, of a kind used ...",48,5,-1,-1
112,"Toilet paper and similar paper, cellulose wadd...",48,-1,-1,-1
292,"Bobbins, spools, cops and similar supports of ...",48,5,-1,-1


In [126]:
for i, cluster in enumerate(clusters):
    df_hscode4.iloc[cluster, df_hscode4.columns.get_loc('cluster_group_id')] = i

In [79]:
df_hscode4

,hscode4_text,HSCode2,group_id,predict_group_id,cluster_group_id
0,ผลไม้ ลูกนัต และส่วนอื่นที่บริโภคได้ของพืชที่ป...,20,2,-1,2
1,รถยนต์และยานยนต์อื่นๆ ที่ออกแบบสำหรับขนส่งบุคค...,87,1,-1,1
2,สารฆ่าแมลง สารฆ่าสัตว์แทะ สารฆ่ารา สารฆ่าวัชพื...,38,-1,-1,-1
3,มอเตอร์ไฟฟ้าและเครื่องกำเนิดไฟฟ้า (ไม่รวมถึงชุ...,85,13,-1,13
5,ส่วนประกอบที่เหมาะสำหรับใช้เฉพาะหรือส่วนใหญ่ใช...,84,1,-1,1
...,...,...,...,...,...
292,"Bobbins, spools, cops and similar supports of ...",48,5,-1,5
294,Pigments (including metallic powders and flake...,32,-1,-1,-1
297,"Aldehydes, whether or not with other oxygen fu...",29,-1,-1,-1
299,Animal or vegetable fats and oils and their fr...,15,-1,-1,-1


#### HDBSCAN + UMAP

In [ ]:
X_umap = umap.UMAP(n_neighbors=5, n_components=5).fit_transform(X)
clusterer = hdbscan.HDBSCAN(alpha=1.0, min_cluster_size=5)
hdbscan_cluster = clusterer.fit_predict(X_umap)
hdbscan_cluster

##### silhouette score

In [109]:
hdbscan_silhouette = metrics.silhouette_score(X_umap, hdbscan_cluster, metric='euclidean')
hdbscan_silhouette

0.3612751

##### Example First 10 rows of each group

In [111]:
df_sentences['hdbscan_group'] = hdbscan_cluster
df_sentences.groupby('hdbscan_group').head(10).sort_values(by=['hdbscan_group'], ascending=False)[:60]

,text,hdbscan_group,dbscan_group
277,Man - made staple fibres,23,2
154,ผลิตภัณฑ์ทางเภสัชกรรมที่ระบุไว้ในหมายเหตุ 4 ใน...,23,2
298,Ceramic products,23,2
49,ผลิตภัณฑ์เซรามิก ...,23,2
57,ผลิตภัณฑ์เบ็ดเตล็ด,23,2
360,Other organo inorganic compounds.,22,2
267,Inorganic chemicals Organic compounds,22,2
255,Organic chemicals,22,2
251,Miscellaneous chemical products,22,2
355,Nitrile-function compounds.,22,2


#### DBSCAN + UMAP

In [114]:
X_umap = umap.UMAP(n_neighbors=5, n_components=5).fit_transform(X)
dbscan_cluster = DBSCAN(eps=0.5, min_samples=2).fit(X_umap)
dbscan_cluster.labels_

array([ 0,  1,  2,  3,  3,  4,  2,  2,  4,  3,  5,  6,  4,  0,  4,  4,  4,
        7,  2,  7,  3,  4,  8,  7,  6,  7,  0,  9, 10,  9,  7,  9, 10, 11,
        3,  4, 12, 13, 12,  7, 10, 14,  3,  7,  0,  9, 15, 10, 16,  2,  7,
       17,  0,  4,  4,  8, 18,  2, 19,  7, 20,  0,  0,  1,  0,  3,  3, 17,
       21,  2,  3, 22, 12,  5, 11,  6,  4, 10, 21,  0, -1, 21,  3, 23, 10,
        3,  3,  3, 21,  7,  7, 17,  7,  3,  4, 13, 22,  2,  4,  7,  6,  7,
        0, 10,  3, 10, 10, 15, 10,  9,  4, 17,  3, 15,  1, 10,  7, 18, 10,
        7,  1,  3, 22, 23,  6, 10,  3, 11,  3,  3, 10,  3, 12, 21, 15, 10,
        3, 12, 13,  1, 12, 12, 15, 15, 12, 16,  1,  7, 10, 11, 11, 10,  0,
       14,  2,  1, 10,  3,  7,  7,  0,  4,  0,  3,  0,  3, 24,  0, 16,  3,
       24, 13,  7,  6, 10, 10,  7,  3,  1, 17, 13, 18,  0, 22,  0,  3, 10,
       10,  3, 18, 10,  3, 16,  3,  3, 10, 10,  6, 12,  7,  0, 19,  7, 21,
        0,  3,  1,  0, 11,  3, 25,  3,  0, 10, 10,  3, 21,  3,  3, 19,  6,
        7,  0,  3, 10, 17

##### silhouette score

In [115]:
dbscan_silhouette = metrics.silhouette_score(X_umap, dbscan_cluster.labels_, metric='euclidean')
dbscan_silhouette

0.3839889

##### Example First 10 rows of each group

In [116]:
df_sentences['dbscan_group'] = dbscan_cluster.labels_
df_sentences.groupby('dbscan_group').head(10).sort_values(by=['dbscan_group'], ascending=False)[:60]

,text,hdbscan_group,dbscan_group
382,Flat-rolled products of iron or non-alloy stee...,-1,27
391,Flat-rolled products of iron or non-alloy stee...,-1,27
356,Flat-rolled products of iron or non-alloy stee...,-1,27
362,Flat-rolled products of iron or non-alloy stee...,-1,27
390,"Flat-rolled products of stainless steel, of a ...",-1,27
234,สิ่งก่อสร้าง (ไม่รวมถึงอาคารสำเร็จรูปตามประเภท...,-1,26
480,Structures (excluding prefabricated buildings ...,-1,26
210,ผ้าที่ใช้กับเตียง โต๊ะ ห้องน้ำและห้องครัว,-1,25
457,"Bed linen, table linen, toilet linen and kitch...",-1,25
414,"Carboys, bottles, flasks, jars, pots, phials, ...",-1,24


: 